<a href="https://colab.research.google.com/github/anushayerram2025/AIML-TASKS/blob/main/TeluguTextClassificationV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import files
import pandas as pd

# Upload the Excel file
uploaded = files.upload()

# Get the uploaded file name
file_name = list(uploaded.keys())[0]

# Print the file name
print(f"Uploaded file: {file_name}")

Saving train_telugu_news.csv to train_telugu_news (1).csv
Uploaded file: train_telugu_news (1).csv


In [ ]:
!pip install indic-nlp-library
from indicnlp.morph import unsupervised_morph
from indicnlp import common


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 13.6 MB/s eta 0:00:00


In [ ]:
!pip install chardet

In [ ]:
import chardet

In [ ]:
with open(file_name, 'rb') as f:
    encoding = chardet.detect(f.read())['encoding']

In [14]:
df = pd.read_csv(file_name,encoding=encoding)
telugu_data_train = df['body'].tolist()
labels_train0 = df['topic'].tolist()



In [8]:
from google.colab import files
import pandas as pd

# Upload the Excel file
uploaded = files.upload()

# Get the uploaded file name
file_name3 = list(uploaded.keys())[0]

# Print the file name
print(f"Uploaded file: {file_name}")

Saving test_telugu_news.csv to test_telugu_news.csv
Uploaded file: train_telugu_news (1).csv


In [13]:
df = pd.read_csv(file_name3)
telugu_data_test = df['body'].tolist()
labels_test0 = df['topic'].tolist()


In [28]:
from indicnlp.tokenize import *
def tokenise(telugu_data,labels):
    tokenised_paragraphs=[]
    lab=[]
    for telugu_sentence,label in zip(telugu_data,labels):
        try:
          x=indic_tokenize.trivial_tokenize(telugu_sentence, lang='te')
          tokenised_paragraphs.append(x)
          lab.append(label)
        except:
          continue

    return (tokenised_paragraphs,lab)


In [29]:
tokenised_paragraphs_train,labels_train=tokenise(telugu_data_train,labels_train0)
tokenised_paragraphs_test,labels_test=tokenise(telugu_data_test,labels_test0)
tokenised_paragraphs=tokenised_paragraphs_train+tokenised_paragraphs_test

In [20]:
print(len(tokenised_paragraphs_test),len(labels_test))
print(len(tokenised_paragraphs_train),len(labels_train))

4329 4329
10243 10243


In [21]:
def vectorData(tokenised_paragraphs,labels):
    X= []
    domains=[]
    for paragraph,label in zip(tokenised_paragraphs,labels):
        vectors = []

        for word in paragraph:
            if word in model.wv.key_to_index:
                vectors.append(model.wv[word])
        if vectors:
            paragraph_vector = np.mean(vectors, axis=0)
            domains.append(label)
            X.append(paragraph_vector)
    return (X,domains)

In [22]:
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np

model = Word2Vec(min_count=1, window=5)

model.build_vocab(tokenised_paragraphs)
model.train(tokenised_paragraphs, total_examples=len(tokenised_paragraphs), epochs=10)

X_train,labels_train1=vectorData(tokenised_paragraphs_train,labels_train)
X_test,labels_test1=vectorData(tokenised_paragraphs_test,labels_test)

y_train=np.array(labels_train1)
y_test=np.array(labels_test1)



In [27]:
knn_classifier = KNeighborsClassifier(n_neighbors=500)
knn_classifier.fit(X_train, y_train)
knn_predictions = knn_classifier.predict(X_test)
knn_accuracy = accuracy_score(y_test, knn_predictions)
print("KNN Accuracy:", knn_accuracy * 100)

KNN Accuracy: 70.73227073227073
